Maxwell solver for Pec (direct ansatz)
=============================

**keys**: Maxwell double layer potential operator, PEC scattering, Dirichlet bvp, Neumann data, MoM, EFIE  

In [ ]:
from netgen.occ import *
import netgen.meshing as meshing
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG

We consider a perfect conductor $\Omega \subset \mathbb R^3$ and a plane wave $\boldsymbol E_{\mathrm{inc}} $ with tangential trace $\gamma_R \boldsymbol E_{\mathrm{inc}} = -\boldsymbol m \in \boldsymbol H^{-\frac12}\left( \mathrm{curl}_\Gamma, \Gamma\right)\,.$ The incoming wave thus induces a scattered electric field $\boldsymbol E$ which propagates into $\Omega^c$. The scattered electric field solves the following Dirichlet boundary value problem: 

$$ \left\{ \begin{array}{rcl l} \mathbf{curl} \, \mathbf{curl}\, \boldsymbol E - \kappa^2 \, \boldsymbol E &=& \boldsymbol 0, \quad &\textnormal{in } \Omega^c \subset \mathbb R^3\,,\\ \gamma_R \,\boldsymbol E &=& \boldsymbol m, \quad & \textnormal{on }\Gamma \\ \left\| \mathbf{curl} \, \boldsymbol E( x) - i\,\omega\,\epsilon \, \boldsymbol E( x)\right\| &=& \mathcal O\left( \displaystyle \frac{1}{\| x\|^2}\right), &\textnormal{for} \; \|x\| \to \infty\,.\end{array} \right. $$ 

and a possible representation reads

$$ \boldsymbol E(x) = \underbrace{\int\limits_\Gamma \displaystyle{ \frac{1}{4\,\pi} \, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \boldsymbol j(y)\, \mathrm{d}\sigma_y + \frac{1}{\kappa^2} \nabla \int\limits_\Gamma \displaystyle{ \frac{1}{4\,\pi}\, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \mathrm{div}_\Gamma \boldsymbol j(y)\, \mathrm{d}\sigma_y }_{\displaystyle{\mathrm{SL}(\boldsymbol j)} } + \underbrace{ \nabla \times \int\limits_\Gamma \displaystyle{ \frac{1}{4\,\pi} \, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \boldsymbol m(y)\, \mathrm{d}\sigma_y }_{\displaystyle{ \mathrm{DL} (\boldsymbol m) } } \,,$$

where $\boldsymbol j = \gamma_N \boldsymbol E$ denotes the Neumann trace and $\boldsymbol m = \gamma_R \boldsymbol E$ the Dirichlet trace of $\boldsymbol E$. 

We carefully apply the tangential trace $\gamma_R$ on $\boldsymbol E$ and obtain a boundary integral equation for unknown Neumann data $\boldsymbol j$. The bounary integral equation is solved by the boundary element method, i.e. the numerical solution of the variational formulation 

$$ \begin{array}{c} \forall \, \boldsymbol v\in H^{-\frac12}(\mathrm{div}_\Gamma, \Gamma): \quad \left\langle \mathrm{SL} (\boldsymbol j), \boldsymbol v \right\rangle_{-\frac12} = \left\langle \boldsymbol m, \boldsymbol v\right\rangle_{-\frac12}  - \left\langle \mathrm{DL}(\boldsymbol m), \boldsymbol v\right\rangle_{-\frac12} \,. \end{array}$$ 

In the enineering community, the approximation scheme is also known as **method of moments** (MoM). 

Define the geometry of the perfect conductor $\Omega$ and create a mesh:

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.3, perfstepsend=meshing.MeshingStep.MESHSURFACE)).Curve(3)

Next, we create finite element spaces for $\boldsymbol H^{-\frac12}(\mathrm{curl}_\Gamma, \Gamma)$ and rotate them later to obtain $\boldsymbol H^{-\frac12}(\mathrm{div}_\Gamma, \Gamma)$ conforming shape functions as test and trial space. 

In [ ]:
fesHCurl = HCurl(mesh, order=2, complex=True)
uHCurl,vHCurl = fesHCurl.TnT() # H(curl_Gamma) conform spaces
print ("ndof HCurl = ", fesHCurl.ndof)

Define the incoming plane wave and compute the given boundary data $\boldsymbol m = -\gamma_R \boldsymbol E_{\mathrm{inc}}$ : 

In [ ]:
eps0 = 8.854e-12 
mu0 = 4*pi*1e-7
omega = 1.5e9
kappa = omega*sqrt(eps0*mu0)
print("kappa = ", kappa)

E_inc = CF((1,0,0))*exp( -1j * kappa * z )

n = specialcf.normal(3)
m = GridFunction(fesHCurl) 
m.Set(-E_inc, definedon=mesh.Boundaries(".*"), dual=True) # -(nxE)xn HCurl-conform (natural trace, PBI operator) 

In [ ]:
Draw (m.real*m.real + m.imag*m.imag, mesh, draw_vol=False, order=2);

The discretisation of the above variational formulation leads to a system of linear equations, ie
$$ \mathrm{V} \, \mathrm j = \left( \frac12 \,\mathrm{Id} - \mathrm{K} \right)\, \mathrm{m}\,,$$
where 
* $\mathrm V$ is the Maxwell single layer operator. $V$ is a regular, symmetric matrix.
* $\mathrm{Id}$ is the mass matrix.
* $\mathrm K$ is the Maxwell double layer operator. $K$ is quadratic. 

In [ ]:
j = GridFunction(fesHCurl) 
pre = BilinearForm(uHCurl.Trace()*vHCurl.Trace()*ds).Assemble().mat.Inverse(freedofs=fesHCurl.FreeDofs()) 
with TaskManager(): 
    V = MaxwellSingleLayerPotentialOperator(fesHCurl, kappa, 
                                            intorder=12, leafsize=40, eta=3., eps=1e-6, method="svd", testhmatrix=False)
    Id = BilinearForm(0.5 * Cross(uHCurl.Trace(),n) * vHCurl.Trace() * ds(bonus_intorder=3)).Assemble() # <H(curl_Gamma), H(div_Gamma)>
    K = MaxwellDoubleLayerPotentialOperator(fesHCurl, kappa, 
                                            intorder=12, leafsize=40, eta=3., eps=1e-6, method="svd", testhmatrix=False)
    rhs = ( (Id.mat - K.mat) * m.vec ).Evaluate() 
    CG(mat = V.mat, pre=pre, rhs = rhs, sol=j.vec, tol=1e-8, maxsteps=500, initialize=False, printrates=False)

In [ ]:
Draw (j.real*j.real+j.imag*j.imag, mesh, draw_vol=False, order=2);

In [ ]:
Draw(j, mesh, draw_vol=False, order=3, min=-3, max=3, animate_complex=True);

**Check the numerical result for $j$ with help of solving the indirect formulation**

The density $\boldsymbol j$ approximates the Neumann trace $\gamma_N \boldsymbol E$ of the scattered field $\boldsymbol E$ and $\boldsymbol j$ is related to the solution $\boldsymbol j_{\mathrm{ind}}$ of the EFIE (indirect formulation). It holds 

$$ \boldsymbol j_{\mathrm{ind}} = \boldsymbol j_{\mathrm{inc}} + \boldsymbol j\,,$$ 

where $\boldsymbol j_{\mathrm{inc}}$ denotes the Neumann trace of the incoming plance wave. The idea is as follows:
- compute $\boldsymbol j_{\mathrm{inc}}$.
- compute $\boldsymbol j_{\mathrm{ind}}$.
- solve the indirect formulation and compare the solutions. Note that the system matrix stays the same. The right hand side vector is different. 

We compute $\boldsymbol j_{\mathrm{ind}}$ and plot the trace:

In [ ]:
curlE_inc = CF((0,-1j*kappa,0))*exp( -1j * kappa * z )
j_inc = GridFunction(fesHCurl) 
j_inc.Set( curlE_inc, definedon=mesh.Boundaries(".*"), dual=True) # dual=True calls the PBI operator 
j_ind = GridFunction(fesHCurl) 
j_ind.Set ( j_inc + j, definedon=mesh.Boundaries(".*"), dual=True) 
Draw (j_ind.real*j_ind.real+j_ind.imag*j_ind.imag, mesh, draw_vol=False, order=2);

We solve the EFIE (see demo **Maxwell_DtN_inidrect**) and plot the trace: 

In [ ]:
test_m = -Cross(n,E_inc) 
test_rhs = LinearForm( Cross(test_m,n) * Cross(n,vHCurl.Trace()) *ds(bonus_intorder=3)).Assemble() 
CG(mat = V.mat, pre=pre, rhs = test_rhs.vec, sol=j_ind.vec, tol=1e-8, maxsteps=500, initialize=False, printrates=False)
Draw (j_ind.real*j_ind.real+j_ind.imag*j_ind.imag, mesh, draw_vol=False, order=3);